In [1]:
# 🔁 Import pipeline components
from regressorpipeline.train import train_fire_model, train_multiple_cnn_for_fire
from regressorpipeline.predict import (
    load_model_bundle,
    predict_fire_risk,
    predict_fire_risk_from_models,
)
from regressorpipeline.visualize import plot_fire_risk_surface_matplotlib

import pandas as pd
import os


In [ ]:
# ✅ Test paths
train_path = "example_data_train.xlsx"
test_path = "example_data_test.xlsx"

from regressorpipeline.train import train_fire_model, train_multiple_cnn_for_fire
from regressorpipeline.predict import (
    load_model_bundle,
    predict_fire_risk,
    predict_fire_risk_from_models,
)
from regressorpipeline.visualize import plot_fire_risk_surface_matplotlib
import pandas as pd
import numpy as np

# 🔧 1. Train and evaluate a single CNN model
model, metrics = train_fire_model("cnn", train_path, save=True)
print("Single CNN model metrics:", metrics)

# 🔧 2. Train CNN ensemble (3 runs) and save
models, metrics_list, ensemble_metrics = train_multiple_cnn_for_fire(
    train_path, n_runs=3, save_path=ensemble_model_path
)
print("Ensemble metrics:", ensemble_metrics)

# 🔍 3. Load and test prediction (single model)
models, scaler_X, scaler_y = load_model_bundle(cnn_model_path)
preds = predict_fire_risk(models[0], scaler_X, scaler_y, test_path)
print("Single CNN Predictions:", preds[:5])

# 🔍 4. Load and test ensemble prediction
models, scaler_X, scaler_y = load_model_bundle(ensemble_model_path)
avg_preds = predict_fire_risk_from_models(models, scaler_X, scaler_y, test_path)
print("Averaged Ensemble Predictions:", avg_preds[:5])

# 🔍 5. Load and test OLS model prediction
models, scaler_X, scaler_y = load_model_bundle(ols_model_path)
ols_preds = predict_fire_risk(models[0], scaler_X, scaler_y, test_path)
print("OLS Predictions:", ols_preds[:5])

# 📊 6. Generate CNN surface plot
df = pd.read_excel(test_path, engine="openpyxl")
X_scaled_df = pd.DataFrame(
    scaler_X.transform(np.log1p(df.select_dtypes(include="number"))),
    columns=df.select_dtypes(include="number").columns
)

plot_fire_risk_surface_matplotlib(
    model=models[0],
    X_scaled_df=X_scaled_df,
    scaler_X=scaler_X,
    scaler_y=scaler_y,
    feat1_name="Thermal Inertia",  # must match feature name
    feat2_name="HRRPUA",           # must match feature name
    title="CNN Prediction Surface",
    save_path="examples/cnn_surface.html"
)
print("✅ Surface plot saved to examples/cnn_surface.html")



[I 2025-05-31 17:28:37,744] A new study created in memory with name: no-name-5186f475-8c3a-4b48-a6fa-c144dd494405
[I 2025-05-31 17:28:38,349] Trial 0 finished with value: 8.489504116466572 and parameters: {'num_filters1': 23, 'num_filters2': 25, 'fc1_size': 50, 'lr': 0.0006287960577413796}. Best is trial 0 with value: 8.489504116466572.
[I 2025-05-31 17:28:38,380] Trial 1 finished with value: 0.8782839909308935 and parameters: {'num_filters1': 26, 'num_filters2': 50, 'fc1_size': 48, 'lr': 0.0010221840455743102}. Best is trial 1 with value: 0.8782839909308935.
[I 2025-05-31 17:28:38,408] Trial 2 finished with value: 0.6243253732383445 and parameters: {'num_filters1': 9, 'num_filters2': 48, 'fc1_size': 65, 'lr': 0.00214961907567656}. Best is trial 2 with value: 0.6243253732383445.
[I 2025-05-31 17:28:38,441] Trial 3 finished with value: 0.16639361476133563 and parameters: {'num_filters1': 31, 'num_filters2': 38, 'fc1_size': 35, 'lr': 0.0005460437933286176}. Best is trial 3 with value: 0.


🔥 Model 'cnn' Evaluation:
R²: 0.6517
MAE: 0.0827
MSE: 0.0119
Single CNN model metrics: {'R²': 0.6516575408325311, 'MAE': 0.082710005, 'MSE': 0.011927585}


[I 2025-05-31 17:28:38,944] Trial 6 finished with value: 8.523072596440215 and parameters: {'num_filters1': 29, 'num_filters2': 47, 'fc1_size': 51, 'lr': 0.0003428172415671805}. Best is trial 2 with value: -0.6309251937180491.
[I 2025-05-31 17:28:38,979] Trial 7 finished with value: 9.102588716820994 and parameters: {'num_filters1': 17, 'num_filters2': 22, 'fc1_size': 35, 'lr': 0.0002623637610563548}. Best is trial 2 with value: -0.6309251937180491.
[I 2025-05-31 17:28:39,012] Trial 8 finished with value: 9.423996468674536 and parameters: {'num_filters1': 8, 'num_filters2': 41, 'fc1_size': 45, 'lr': 0.0012327729392905662}. Best is trial 2 with value: -0.6309251937180491.
[I 2025-05-31 17:28:39,041] Trial 9 finished with value: 10.283025905194998 and parameters: {'num_filters1': 17, 'num_filters2': 24, 'fc1_size': 88, 'lr': 0.00018086340540920094}. Best is trial 2 with value: -0.6309251937180491.
[I 2025-05-31 17:28:39,068] A new study created in memory with name: no-name-ae1f36e2-2db7-

FileNotFoundError: [Errno 2] No such file or directory: ''